In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import itertools
from joblib import Parallel, delayed
from numba import njit
from Utils import extract_symbol_timeframe

# === Numba-реализация EMA и RSI ===
@njit
def ema(arr, window):
    alpha = 2 / (window + 1)
    result = np.empty_like(arr)
    result[0] = arr[0]
    for i in range(1, len(arr)):
        result[i] = alpha * arr[i] + (1 - alpha) * result[i - 1]
    return result

@njit
def compute_rsi(close, period):
    delta = np.diff(close)
    up = np.where(delta > 0, delta, 0.0)
    down = np.where(delta < 0, -delta, 0.0)

    roll_up = np.empty_like(close)
    roll_down = np.empty_like(close)

    roll_up[0] = 0.0
    roll_down[0] = 0.0

    roll_up[1] = np.mean(up[:period])
    roll_down[1] = np.mean(down[:period])

    for i in range(2, len(close)):
        roll_up[i] = (roll_up[i-1] * (period - 1) + up[i-1]) / period
        roll_down[i] = (roll_down[i-1] * (period - 1) + down[i-1]) / period

    rs = roll_up / (roll_down + 1e-10)
    rsi = 100.0 - (100.0 / (1.0 + rs))
    rsi[:period] = np.nan
    return rsi

# === Расчёт по одной комбинации ===
def process_combination(period, entry_threshold, tp, close_all, high_all, low_all, time_all):
    rsi = compute_rsi(close_all, period)
    cross_up = (rsi[:-1] < entry_threshold) & (rsi[1:] >= entry_threshold)
    entry_idx = np.where(cross_up)[0] + 1

    balance = 10.0
    durations, entry_times = [], []
    num_trades, last_exit = 0, 0
    stop_triggered = False
    first_stop_time = pd.NaT
    stop_loss_pct = 99

    for i in entry_idx:
        if i <= last_exit or i >= len(close_all):
            continue

        entry_price = close_all[i]
        take_profit = entry_price * (1 + tp / 100)
        stop_loss = entry_price * (1 - stop_loss_pct / 100)
        entry_time = time_all[i]

        for j in range(i + 1, len(close_all)):
            if high_all[j] >= take_profit:
                exit_price = take_profit
                exit_time = time_all[j]
                break
            elif low_all[j] <= stop_loss:
                exit_price = stop_loss
                exit_time = time_all[j]
                if not stop_triggered:
                    stop_triggered = True
                    first_stop_time = exit_time
                break
        else:
            break

        profit_pct = (exit_price / entry_price - 1) * 100
        balance *= (1 + profit_pct / 100)
        duration = int((exit_time - entry_time).astype("timedelta64[m]") / np.timedelta64(1, 'm'))

        durations.append(duration)
        entry_times.append(entry_time)
        num_trades += 1
        last_exit = j

    if num_trades == 0:
        return None

    return {
        "rsi_period": period,
        "entry_threshold": entry_threshold,
        "tp_range": tp,
        "profit_total_pct": round((balance / 10 - 1) * 100, 2),
        "num_trades": num_trades,
        "min_time": int(np.min(durations)),
        "max_time": int(np.max(durations)),
        "mean_time": round(np.mean(durations), 2),
        "median_time": int(np.median(durations)),
        "last_entry_time": entry_times[-1],
        "stop_triggered": stop_triggered,
        "stop_time": first_stop_time if stop_triggered else pd.NaT
    }

# === Загрузка данных ===
path = " " # Format filename = {symbol}USDT_timeframe_data.csv, example: "BTCUSDT_1h_from_1_Jan_2020.csv"
df = pd.read_csv(path)

df["open_time"] = pd.to_datetime(df["open_time"], utc=True)
close_all = df["close"].to_numpy(dtype=np.float32)
high_all = df["high"].to_numpy(dtype=np.float32)
low_all = df["low"].to_numpy(dtype=np.float32)
time_all = df["open_time"].to_numpy(dtype="datetime64[m]")

# === Параметры перебора ===
symbol, timeframe = extract_symbol_timeframe(path)
indicator_name = "RSI_simple"

rsi_range = range(5, 51, 2)
tp_range = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2, 1.5, 1.7, 2, 2.5, 3] + list(range(4, 11))
entry_thresholds = [10, 15, 20, 23, 25, 27, 30, 35]

combos = list(itertools.product(rsi_range, entry_thresholds, tp_range))

# === Параллельный запуск
results = Parallel(n_jobs=-1)(
    delayed(process_combination)(rsi_period, threshold, tp, close_all, high_all, low_all, time_all)
    for rsi_period, threshold, tp in tqdm(combos, desc="⚙️ Grid Search RSI")
)

# === Финализация и сохранение
results = [r for r in results if r is not None]
result_df = pd.DataFrame(results).sort_values("profit_total_pct", ascending=False)

out_name = f"{symbol}_{timeframe}_{indicator_name}_gridsearch_{len(combos)}_combos.csv"
result_df.to_csv(out_name, index=False)

print(f"📁 Результаты сохранены в: {out_name}")
print(f"🏆 Топ-5 лучших стратегий для {symbol} на таймфрейме {timeframe}:")
result_df.head(5)
